purpose of this script is to process Google Form Responses from pool play and output record info (wins/losses, win ratio/loss ratio, points) and output with pool/court#, ranking info, team name, and record info

In [178]:
from ReadWriteGDocs import write_gdoc, read_gdoc_range, write_gdoc_cell

m_num_teams = 45
w_num_teams = 45

import pandas as pd
import numpy as np

input_ws = ["NY Mini 2018 Score Input - Men's (Responses)", "NY Mini 2018 Score Input - Women's (Responses)",
           "Form Values Add-on - DO NOT DELETE"]
output_ws = ["NY Mini 2018 - 1st Round", "NY Mini 2018 - 2nd Round"]

c=read_gdoc_range(input_ws[0], "Form Responses 1", 1,2,1,12)
c=c[0]

# get list of team names from Form Values
results = pd.DataFrame(read_gdoc_range(input_ws[2], "Sheet1", 2, 4, 1+m_num_teams, 4), columns=["Team Name"])
results["Court Number"] = ""
results["Wins"] = 0
results["Points"] = 0

# get court numbers from master day 1
courts = pd.DataFrame(read_gdoc_range(output_ws[0], "Men's Master Day 1", 3, 2, 9, 10))

# iterate each row
for i in range(1,courts.shape[0]-1):
    for j in range(0, courts.shape[1]):
        team_name = courts.at[i, j]
        
        if (team_name != "X")
            team_row = results.loc[results["Team Name"] == team_name]
            results.at[team_row.index, "Court Number"] = courts.at[0, j]
        

# start processing from row 2
# keep track of rows processed using processed

processed = 0

results

,Team Name,Court Number,Wins,Points
0,BOS Freemason,CT02,0,0
1,BOS Hurricanes Black,CT08,0,0
2,BOS Hurricanes Blue,CT01,0,0
3,BOS Knights A,CT14,0,0
4,BOS Knights B,CT12,0,0
5,BOS Knights C,CT13,0,0
6,BOS Knights X,CT03,0,0
7,BOS Rising Tide,CT07,0,0
8,CHI Chicago United,CT08,0,0
9,Montreal Freemason,CT07,0,0


In [179]:
# m_in = pd.DataFrame(read_gdoc_range(input_ws[0], "Form Responses 1", processed + 2,2, 100, 12), 
#                     columns=c)

m_in = pd.DataFrame(read_gdoc_range(input_ws[0], "Form Responses 1", processed + 2,2, 200, 12), 
                    columns=c)

m_in.replace('', np.nan, inplace=True)
m_in.dropna(inplace=True, how='all')

from math import isnan

print ("Need to process " + str(len(m_in)) + " games")

Need to process 89 games


In [180]:
m_in
m_dropped = m_in.drop(c[2:5], axis=1)
## Convert to numeric
for i in m_dropped.columns:
    m_dropped[i] = pd.to_numeric(m_dropped[i], downcast='integer', errors='ignore')
    
# pull in list of team names - add columns for wins/points - "results"
# court number, team name, wins, losses, points

# iterate through each row of m_dropped and see if team name is found

for index,row in m_dropped.iterrows():
    team_1_name = row["Team 1 Name"]
    team_2_name = row["Team 2 Name"]
    win1 = 0
    win2 = 0
    points1 = 0
    points2 = 0
    
    # process first set
    score_diff = row["Team 1 Score"] - row["Team 2 Score"]
        
    if (score_diff > 0):
        win1 += 1
    elif (score_diff < 0):
        win2 += 1
    ## if score_diff == 0 then do nothing
        
    # process second set
    score_diff = row["Team 1 Score 2"] - row["Team 2 Score 2"]
    points1 += score_diff
    points2 -= score_diff
    
    if (score_diff > 0): # team 1 won
        win1 += 1
    elif (score_diff < 0): # team 2 won
        win2 += 1
    ## if score_diff == 0 then do nothing
                    
    team_1_row = results.loc[results["Team Name"] == team_1_name]
    results.at[team_1_row.index, "Wins"] = team_1_row["Wins"] + win1
    results.at[team_1_row.index, "Points"] = team_1_row["Points"] + points1
    
    team_2_row = results.loc[results["Team Name"] == team_2_name]
    results.at[team_2_row.index, "Wins"] = team_2_row["Wins"] + win2
    results.at[team_2_row.index, "Points"] = team_2_row["Points"] + points2
    
# add to results: wins and points

    processed = processed + 1
        
print("Number of records processed (men's): " + str(len(m_in)))

print("Total Processed: " + str(processed))

Number of records processed (men's): 89
Total Processed: 89


In [181]:
## still need to process into second day pool plays
## get overall ranking - criteria is by ranking
## snake teams into second day pool play

# sort by court number, wins, points for seeding
results = results.sort_values(by=["Court Number", "Wins", "Points"], ascending=[1,0,0])

,Team Name,Court Number,Wins,Points
19,NY Strangers B,CT01,7,61
28,Toronto Connex A,CT01,5,61
43,Wash DC Yee Fung Toy Lightning,CT01,3,11
2,BOS Hurricanes Blue,CT01,3,-31
37,Wash DC CYC C,CT01,0,-102
33,Toronto Ngun Lam Red,CT02,8,81
41,Wash DC MVP B,CT02,5,5
30,Toronto Connex C,CT02,4,-9
11,NY BCVA,CT02,3,-25
0,BOS Freemason,CT02,0,-52


In [198]:
# sort by court number and team name

disp_results = results.sort_values(by=["Court Number", "Team Name"]).reset_index(drop=True)

# update display output - court numbers and team names - one time only
# for i in range(0, len(disp_results)):
#     write_gdoc_cell(disp_results.at[i, "Court Number"], output_ws[0], "ScoreDisp", 5+i, 2)
#     write_gdoc_cell(disp_results.at[i, "Team Name"], output_ws[0], "ScoreDisp", 5+i, 3)
    
# update display output - wins and points - keep updating
for i in range(0, len(disp_results)):
    write_gdoc_cell(int(disp_results.at[i, "Wins"]), output_ws[0], "ScoreDisp", 5+i, 4)
    write_gdoc_cell(int(disp_results.at[i, "Points"]), output_ws[0], "ScoreDisp", 5+i, 5)